In [1]:
import numpy as np
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, LeavePOut, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder, minmax_scale, MinMaxScaler, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree, export_text
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout, Activation, Flatten
from sklearn.metrics import (accuracy_score,confusion_matrix, ConfusionMatrixDisplay, r2_score, silhouette_score, 
balanced_accuracy_score,cohen_kappa_score, recall_score, precision_score, f1_score)
from kneed import knee_locator, KneeLocator
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("E:/Python Programs/new_y1.csv")
df.head()

,Date,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,1st Num
0,1/1/2021,Friday,20,40,9,2,80,84,72,84,5,54,16
1,5/1/2021,Tuesday,30,60,26,2,42,84,19,84,5,54,12
2,8/1/2021,Friday,40,80,48,3,63,22,38,84,5,54,18
3,12/1/2021,Tuesday,11,21,80,3,22,22,76,84,5,54,2
4,15/1/2021,Friday,11,42,92,4,66,22,20,84,5,54,4


In [3]:
df.drop(labels=['Date'], axis=1, inplace=True)

In [4]:
df.head()

,Weekday,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,1st Num
0,Friday,20,40,9,2,80,84,72,84,5,54,16
1,Tuesday,30,60,26,2,42,84,19,84,5,54,12
2,Friday,40,80,48,3,63,22,38,84,5,54,18
3,Tuesday,11,21,80,3,22,22,76,84,5,54,2
4,Friday,11,42,92,4,66,22,20,84,5,54,4


In [5]:
X = df.iloc[:,0:11]
y = df.iloc[:,-1]

In [6]:
y.shape

(373,)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=20)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((298, 11), (298,), (75, 11), (75,))

In [8]:
ohe = OneHotEncoder(drop='first',dtype=np.int32)

In [9]:
X_train_new = ohe.fit_transform(X_train[['Weekday']]).toarray()
X_test_new = ohe.transform(X_test[['Weekday']]).toarray()

In [10]:
X_train_trf = pd.DataFrame(np.hstack((X_train.iloc[:,1:11].values, X_train_new)), columns=['Lagna','Sun','Moon','Mars','Mercury','Jupiter','Venus','Saturn','Rahu','Ketu','Weekday'])

In [11]:
X_train_trf.head()

,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,Weekday
0,24,23,22,12,42,78,25,69,2,45,1
1,16,14,60,14,39,46,51,22,12,34,1
2,42,81,24,69,1,75,24,46,6,64,0
3,72,20,21,12,36,104,18,46,6,16,1
4,34,12,8,78,6,78,4,69,3,48,0


In [12]:
X_test_trf = pd.DataFrame(np.hstack((X_test.iloc[:,1:11].values, X_test_new)),columns=['Lagna','Sun','Moon','Mars','Mercury','Jupiter','Venus','Saturn','Rahu','Ketu','Weekday'])

In [13]:
X_test_trf.head()

,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,Weekday
0,6,13,56,5,24,52,36,23,6,16,1
1,28,32,48,45,60,46,76,44,12,34,0
2,4,26,8,42,36,6,10,24,1,42,1
3,36,69,18,60,84,72,60,88,6,64,1
4,22,84,33,38,22,48,40,88,9,17,0


In [14]:
model = LinearRegression()
model.fit(X_train_trf, y_train)

LinearRegression()

In [15]:
model.score(X_test_trf, y_test)

-0.09941438112713752

In [16]:
robsc = RobustScaler()

In [17]:
X_train_rbtrf = robsc.fit_transform(X_train_trf)

In [18]:
X_test_rbtrf = robsc.transform(X_test_trf)

In [19]:
X_train_rbtrf = pd.DataFrame(X_train_rbtrf, columns=['Lagna','Sun','Moon','Mars','Mercury','Jupiter','Venus','Saturn','Rahu','Ketu','Weekday'])
X_train_rbtrf

,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,Weekday
0,-0.212121,-0.027778,-0.136054,-0.235294,0.531469,0.796935,-0.027778,0.500000,-0.500,0.250000,1.0
1,-0.454545,-0.277778,0.897959,-0.176471,0.447552,0.306513,0.694444,-0.619048,0.750,-0.208333,1.0
2,0.333333,1.583333,-0.081633,1.441176,-0.615385,0.750958,-0.055556,-0.047619,0.000,1.041667,0.0
3,1.242424,-0.111111,-0.163265,-0.235294,0.363636,1.195402,-0.222222,-0.047619,0.000,-0.958333,1.0
4,0.090909,-0.333333,-0.517007,1.705882,-0.475524,0.796935,-0.611111,0.500000,-0.375,0.375000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
293,0.242424,0.027778,0.353741,2.000000,1.454545,-0.337165,0.555556,0.571429,6.000,0.541667,1.0
294,-0.575758,1.777778,-0.163265,-0.235294,1.594406,0.796935,1.277778,0.500000,-0.500,0.250000,0.0
295,-0.545455,0.000000,1.986395,-0.117647,1.818182,1.195402,0.722222,1.047619,-0.250,-0.375000,1.0
296,0.393939,-0.166667,-0.517007,0.705882,-0.027972,-0.306513,0.277778,0.000000,-0.500,0.708333,1.0


In [20]:
X_test_rbtrf = pd.DataFrame(X_test_rbtrf, columns=['Lagna','Sun','Moon','Mars','Mercury','Jupiter','Venus','Saturn','Rahu','Ketu','Weekday'])
X_test_rbtrf.head()

,Lagna,Sun,Moon,Mars,Mercury,Jupiter,Venus,Saturn,Rahu,Ketu,Weekday
0,-0.757576,-0.305556,0.789116,-0.441176,0.027972,0.398467,0.277778,-0.595238,0.000,-0.958333,1.0
1,-0.090909,0.222222,0.571429,0.735294,1.034965,0.306513,1.388889,-0.095238,0.750,-0.208333,0.0
2,-0.818182,0.055556,-0.517007,0.647059,0.363636,-0.306513,-0.444444,-0.571429,-0.625,0.125000,1.0
3,0.151515,1.250000,-0.244898,1.176471,1.706294,0.704981,0.944444,0.952381,0.000,1.041667,1.0
4,-0.272727,1.666667,0.163265,0.529412,-0.027972,0.337165,0.388889,0.952381,0.375,-0.916667,0.0


In [21]:
model.fit(X_train_rbtrf, y_train)

LinearRegression()

In [22]:
model.score(X_test_rbtrf, y_test)

-0.0994143811271373

In [23]:
model_lt = LogisticRegression()
model_lt.fit(X_train_rbtrf, y_train)

LogisticRegression()

In [24]:
model_lt.score(X_test_rbtrf, y_test)

0.10666666666666667

In [29]:
# ProfileReport(df=df, title="E:/Python Programs/EuroAstro/newastro_ouput1.html", explorative=True)